In [8]:
import math
import copy
fread = open("net1.txt", "r") #netdata.txt
G_v_e = fread.readline()
G_v_e = G_v_e.split(' ') #change to \t if tab spaced
V = int(G_v_e[0])
E = int(G_v_e[1].strip())
V_list = list(range(0,V,1))
lines = fread.readlines()
A_dict = {}
G_mat = [ [ 0 for i in range(4) ] for j in range(E) ]
g = 0
for line in lines:
    t_h = line.split('\t') #change to \t if tab spaced
    print(t_h)
    G_mat[g][0] = int(t_h[0])
    G_mat[g][1] = int(t_h[1])
    G_mat[g][2] = int(t_h[2])
    G_mat[g][3] = int(t_h[3].strip())
    g += 1
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    if tail in A_dict.keys():
        A_dict[tail].append(head)        
    else:
        A_dict[tail] = [head]

capacity = [[0 for i in range(V)] for j in range(V)]
cost = [[math.inf for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    cost[tail][head] = G_mat[i][2]
    capacity[tail][head] = G_mat[i][3]

print(G_mat, "\n")
print(A_dict)
print(cost)
print(capacity)
fread.close()

['0', '1', '461', '765\n']
['0', '21', '826', '814\n']
['1', '0', '716', '923\n']
['1', '2', '324', '806\n']
['1', '21', '713', '789\n']
['2', '1', '826', '649\n']
['2', '3', '423', '628\n']
['2', '21', '533', '263\n']
['3', '2', '284', '100\n']
['3', '23', '639', '878\n']
['3', '24', '208', '288\n']
['4', '3', '370', '601\n']
['4', '5', '359', '546\n']
['4', '24', '565', '745\n']
['4', '23', '223', '792\n']
['5', '6', '912', '404\n']
['5', '24', '863', '168\n']
['5', '26', '957', '960\n']
['6', '5', '328', '625\n']
['6', '25', '974', '790\n']
['7', '27', '263', '962\n']
['7', '26', '702', '103\n']
['7', '28', '816', '628\n']
['8', '9', '133', '868\n']
['8', '28', '390', '256\n']
['8', '27', '732', '967\n']
['9', '10', '607', '828\n']
['9', '28', '584', '278\n']
['9', '30', '226', '500\n']
['10', '9', '261', '776\n']
['10', '11', '548', '922\n']
['10', '29', '951', '900\n']
['10', '31', '131', '535\n']
['11', '12', '930', '901\n']
['11', '31', '416', '484\n']
['11', '30', '556', '632\n

In [9]:
def create_residual_network(A_dict, capacity, cost, V):
    R = []
    enode = V
    visited = [[0 for i in range(V)] for j in range(V)]
    for i in A_dict.keys():
        for j in A_dict[i]:
            #print(i, j)
            if (j in A_dict.keys()):
                if (visited[i][j] == 0):
                    if (i in A_dict[j]):
                        #add two nodes
                        R.append([i, enode, cost[i][j], capacity[i][j]])
                        #R.append([enode, i, capacity[i][j], cost[i][j]])
                        R.append([enode, j, 0, capacity[i][j]])
                        enode = enode + 1
                        R.append([j, enode, cost[j][i], capacity[j][i]])
                        R.append([enode, i, 0, capacity[j][i]]) 
                        enode = enode + 1
                    else:
                        R.append([i, j,  cost[i][j], capacity[i][j]])
                else:
                    continue
            else:
                R.append([i, j,  cost[i][j], capacity[i][j]])
            #print(R)
            visited[i][j] = 1
            visited[j][i] = 1
    #print(R)
    itr = len(R)
    #print(itr)
    for i in range(itr):
        t = R[i][0]
        h = R[i][1]
        ncost = -1*R[i][2]
        R.append([h, t, ncost, 0])
    return R, enode

In [10]:
def retrieve_flow_1(R_orig, R_capacity):
    flow = []
    for i in range(len(R_orig)):
        t = R_orig[i][0]
        h = R_orig[i][1]
        old = R_orig[i][3]
        new = R_capacity[t][h]
        if (old > new):
            flow.append(old - new)
        else:
            flow.append(0)            
    return flow


# Develop modified dictionary
def mod_dict(R, capacity):
    R_dict = {}
    for e in range(len(R)):
        t = R[e][0]
        h = R[e][1]
        if (capacity[t][h] != 0):
            if (t in R_dict.keys()):
                if (h not in R_dict[t]):
                    R_dict[t].append(h)
            else:
                R_dict[t] = [h]
        else:
            if (t in R_dict.keys()):
                if (h in R_dict[t]):
                    R_dict[t].remove(h)
    return R_dict


In [11]:
def pc_Bellman_Ford(R, V, s, t, R_capacity):
    parent = {}
    for i in range(V):
        parent[i] = i
    #print(parent)
    dist = [math.inf]*V
    dist[s] = 0
    changed_flag = True
    itr = 1
    while (changed_flag):
        changed_flag = False
        d_copy = dist.copy()  #remove this for modified
        for i in range(len(R)):
            tail = R[i][0]
            head = R[i][1]
            if (d_copy[head] > dist[tail] + R[i][2] and R_capacity[tail][head] > 0): #d_copy[head] to dist[head]
                d_copy[head] = dist[tail] + R[i][2] #d_copy[head] to dist[head]
                parent[head] = tail
                changed_flag = True
        dist = d_copy #remove this for modified
        #print(itr)
        #print(dist)
        #print(parent)
        itr = itr + 1
        if (itr > V): #to take care of negative cycles
            break    
    path = [t]
    curr = t
    while (curr != s and curr != parent[curr]):
        curr = parent[curr]
        path.insert(0, curr)
    if (path[0] != s):
        path = []
    return dist[t], path

In [12]:
#capacity scaling
def capacity_scaling(R, V, R_capacity, R_cost, s, t, f):
    R_orig = copy.deepcopy(R)
    flow_cost = 0
    remaining = f
    dist, path = pc_Bellman_Ford(R, V, s, t, R_capacity)
    flow = []
    while (len(path)!= 0 and remaining > 0):
        cmin = remaining
        for i in range(len(path)-1):
            tail = path[i]
            head = path[i+1]
            if (R_capacity[tail][head] < cmin):
                cmin = R_capacity[tail][head]
        remaining = remaining - cmin
        flow_cost = flow_cost + cmin*dist
        
        #update capacity
        for i in range(len(path)-1):
            tail = path[i]
            head = path[i+1]
            R_capacity[tail][head] =  R_capacity[tail][head] - cmin
            R_capacity[head][tail] =  R_capacity[head][tail] + cmin
        
        # if remaining left
        if (remaining > 0):
            dist, path = pc_Bellman_Ford(R, V, s, t, R_capacity)
    
    if (remaining == 0):
        flow = retrieve_flow_1(R_orig, R_capacity)
        return flow, flow_cost
    else:
        print("Can not send flow f")
        return flow, math.inf()
        
            
        
            
            
        
    

In [13]:
s = 50
t = 340
capacity = [[0 for i in range(V)] for j in range(V)]
cost = [[math.inf for i in range(V)] for j in range(V)]
for i in range(E):
    tail = G_mat[i][0]
    head = G_mat[i][1]
    cost[tail][head] = G_mat[i][2]
    capacity[tail][head] = G_mat[i][3]
    
R, Vnew = create_residual_network(A_dict, capacity, cost, V)

#print(Vnew)
R_capacity = [[0 for i in range(Vnew)] for j in range(Vnew)]
R_cost = [[math.inf for i in range(Vnew)] for j in range(Vnew)]
for i in range(len(R)):
    tail = R[i][0]
    head = R[i][1]
    R_cost[tail][head] = R[i][2]
    R_capacity[tail][head] = R[i][3]

for i in range(len(R)):
    print(R[i])
R_dict = mod_dict(R, R_capacity)
print(R_dict)

[0, 400, 461, 765]
[400, 1, 0, 765]
[1, 401, 716, 923]
[401, 0, 0, 923]
[0, 21, 826, 814]
[1, 402, 324, 806]
[402, 2, 0, 806]
[2, 403, 826, 649]
[403, 1, 0, 649]
[1, 21, 713, 789]
[2, 404, 423, 628]
[404, 3, 0, 628]
[3, 405, 284, 100]
[405, 2, 0, 100]
[2, 21, 533, 263]
[3, 406, 639, 878]
[406, 23, 0, 878]
[23, 407, 818, 972]
[407, 3, 0, 972]
[3, 408, 208, 288]
[408, 24, 0, 288]
[24, 409, 710, 415]
[409, 3, 0, 415]
[4, 3, 370, 601]
[4, 5, 359, 546]
[4, 24, 565, 745]
[4, 23, 223, 792]
[5, 410, 912, 404]
[410, 6, 0, 404]
[6, 411, 328, 625]
[411, 5, 0, 625]
[5, 412, 863, 168]
[412, 24, 0, 168]
[24, 413, 258, 539]
[413, 5, 0, 539]
[5, 414, 957, 960]
[414, 26, 0, 960]
[26, 415, 172, 694]
[415, 5, 0, 694]
[6, 416, 974, 790]
[416, 25, 0, 790]
[25, 417, 215, 893]
[417, 6, 0, 893]
[7, 27, 263, 962]
[7, 418, 702, 103]
[418, 26, 0, 103]
[26, 419, 598, 739]
[419, 7, 0, 739]
[7, 28, 816, 628]
[8, 9, 133, 868]
[8, 420, 390, 256]
[420, 28, 0, 256]
[28, 421, 791, 538]
[421, 8, 0, 538]
[8, 27, 732, 967]

[759, 173, -899, 0]
[153, 759, 0, 0]
[134, 153, -718, 0]
[760, 153, -155, 0]
[174, 760, 0, 0]
[761, 174, -464, 0]
[153, 761, 0, 0]
[762, 154, -241, 0]
[155, 762, 0, 0]
[763, 155, -757, 0]
[154, 763, 0, 0]
[174, 154, -993, 0]
[764, 154, -480, 0]
[173, 764, 0, 0]
[765, 173, -454, 0]
[154, 765, 0, 0]
[766, 154, -559, 0]
[175, 766, 0, 0]
[767, 175, -955, 0]
[154, 767, 0, 0]
[134, 155, -881, 0]
[768, 155, -482, 0]
[174, 768, 0, 0]
[769, 174, -889, 0]
[155, 769, 0, 0]
[770, 155, -915, 0]
[176, 770, 0, 0]
[771, 176, -965, 0]
[155, 771, 0, 0]
[136, 156, -696, 0]
[155, 156, -224, 0]
[176, 156, -716, 0]
[137, 156, -603, 0]
[175, 156, -155, 0]
[177, 156, -591, 0]
[156, 157, -673, 0]
[158, 157, -806, 0]
[772, 157, -904, 0]
[177, 772, 0, 0]
[773, 177, -554, 0]
[157, 773, 0, 0]
[138, 157, -506, 0]
[176, 157, -851, 0]
[774, 157, -828, 0]
[178, 774, 0, 0]
[775, 178, -769, 0]
[157, 775, 0, 0]
[138, 158, -634, 0]
[178, 158, -392, 0]
[776, 158, -294, 0]
[179, 776, 0, 0]
[777, 179, -598, 0]
[158, 777, 0, 

In [14]:
f = 750 #input flow
flow, flow_cost = capacity_scaling(R, Vnew, R_capacity, R_cost, s, t, f)
# Flow_cost
print('Total flow cost: ', flow_cost)
# Flow thru arcs
print('Flow through arcs-')
flow_value = 0
for i in range(len(R)):
    print('(', R[i][0], ',' , R[i][1], ')', ':', flow[i])

Total flow cost:  6655305
Flow through arcs-
( 0 , 400 ) : 0
( 400 , 1 ) : 0
( 1 , 401 ) : 0
( 401 , 0 ) : 0
( 0 , 21 ) : 0
( 1 , 402 ) : 0
( 402 , 2 ) : 0
( 2 , 403 ) : 0
( 403 , 1 ) : 0
( 1 , 21 ) : 0
( 2 , 404 ) : 0
( 404 , 3 ) : 0
( 3 , 405 ) : 0
( 405 , 2 ) : 0
( 2 , 21 ) : 0
( 3 , 406 ) : 0
( 406 , 23 ) : 0
( 23 , 407 ) : 0
( 407 , 3 ) : 0
( 3 , 408 ) : 0
( 408 , 24 ) : 0
( 24 , 409 ) : 0
( 409 , 3 ) : 0
( 4 , 3 ) : 0
( 4 , 5 ) : 0
( 4 , 24 ) : 0
( 4 , 23 ) : 0
( 5 , 410 ) : 0
( 410 , 6 ) : 0
( 6 , 411 ) : 0
( 411 , 5 ) : 0
( 5 , 412 ) : 0
( 412 , 24 ) : 0
( 24 , 413 ) : 0
( 413 , 5 ) : 0
( 5 , 414 ) : 0
( 414 , 26 ) : 0
( 26 , 415 ) : 0
( 415 , 5 ) : 0
( 6 , 416 ) : 0
( 416 , 25 ) : 0
( 25 , 417 ) : 0
( 417 , 6 ) : 0
( 7 , 27 ) : 0
( 7 , 418 ) : 0
( 418 , 26 ) : 0
( 26 , 419 ) : 0
( 419 , 7 ) : 0
( 7 , 28 ) : 0
( 8 , 9 ) : 0
( 8 , 420 ) : 0
( 420 , 28 ) : 0
( 28 , 421 ) : 0
( 421 , 8 ) : 0
( 8 , 27 ) : 0
( 9 , 422 ) : 0
( 422 , 10 ) : 0
( 10 , 423 ) : 0
( 423 , 9 ) : 0
( 9 , 424

( 734 , 160 ) : 0
( 160 , 735 ) : 0
( 735 , 141 ) : 0
( 142 , 141 ) : 0
( 143 , 163 ) : 0
( 144 , 124 ) : 0
( 144 , 143 ) : 0
( 144 , 736 ) : 0
( 736 , 165 ) : 0
( 165 , 737 ) : 0
( 737 , 144 ) : 0
( 145 , 144 ) : 0
( 145 , 126 ) : 0
( 145 , 738 ) : 197
( 738 , 164 ) : 197
( 164 , 739 ) : 0
( 739 , 145 ) : 0
( 146 , 166 ) : 0
( 146 , 125 ) : 0
( 146 , 127 ) : 0
( 146 , 167 ) : 0
( 147 , 146 ) : 0
( 147 , 126 ) : 0
( 147 , 128 ) : 0
( 148 , 147 ) : 0
( 148 , 740 ) : 0
( 740 , 149 ) : 0
( 149 , 741 ) : 0
( 741 , 148 ) : 0
( 149 , 742 ) : 0
( 742 , 150 ) : 0
( 150 , 743 ) : 0
( 743 , 149 ) : 0
( 149 , 744 ) : 0
( 744 , 169 ) : 0
( 169 , 745 ) : 0
( 745 , 149 ) : 0
( 149 , 130 ) : 0
( 149 , 746 ) : 244
( 746 , 168 ) : 244
( 168 , 747 ) : 0
( 747 , 149 ) : 0
( 150 , 748 ) : 0
( 748 , 170 ) : 0
( 170 , 749 ) : 0
( 749 , 150 ) : 0
( 150 , 169 ) : 309
( 150 , 750 ) : 0
( 750 , 171 ) : 0
( 171 , 751 ) : 0
( 751 , 150 ) : 0
( 151 , 150 ) : 0
( 151 , 752 ) : 0
( 752 , 171 ) : 0
( 171 , 753 ) : 0


( 225 , 224 ) : 0
( 225 , 206 ) : 0
( 226 , 206 ) : 0
( 226 , 225 ) : 0
( 226 , 227 ) : 0
( 226 , 920 ) : 0
( 920 , 246 ) : 0
( 246 , 921 ) : 0
( 921 , 226 ) : 0
( 226 , 207 ) : 0
( 227 , 207 ) : 0
( 227 , 228 ) : 0
( 227 , 247 ) : 175
( 227 , 248 ) : 0
( 228 , 229 ) : 0
( 228 , 248 ) : 0
( 228 , 922 ) : 0
( 922 , 249 ) : 0
( 249 , 923 ) : 0
( 923 , 228 ) : 0
( 229 , 209 ) : 0
( 229 , 208 ) : 0
( 229 , 924 ) : 0
( 924 , 248 ) : 0
( 248 , 925 ) : 0
( 925 , 229 ) : 0
( 230 , 926 ) : 0
( 926 , 231 ) : 0
( 231 , 927 ) : 0
( 927 , 230 ) : 0
( 230 , 928 ) : 0
( 928 , 250 ) : 0
( 250 , 929 ) : 0
( 929 , 230 ) : 0
( 230 , 211 ) : 0
( 230 , 930 ) : 0
( 930 , 251 ) : 0
( 251 , 931 ) : 0
( 931 , 230 ) : 0
( 231 , 251 ) : 0
( 231 , 932 ) : 0
( 932 , 250 ) : 0
( 250 , 933 ) : 0
( 933 , 231 ) : 0
( 231 , 252 ) : 0
( 232 , 231 ) : 0
( 232 , 934 ) : 0
( 934 , 252 ) : 0
( 252 , 935 ) : 0
( 935 , 232 ) : 0
( 233 , 232 ) : 0
( 233 , 234 ) : 0
( 233 , 936 ) : 0
( 936 , 253 ) : 0
( 253 , 937 ) : 0
( 937 , 

( 313 , 1110 ) : 0
( 1110 , 334 ) : 0
( 334 , 1111 ) : 0
( 1111 , 313 ) : 0
( 314 , 1112 ) : 0
( 1112 , 315 ) : 0
( 315 , 1113 ) : 0
( 1113 , 314 ) : 0
( 314 , 1114 ) : 0
( 1114 , 334 ) : 0
( 334 , 1115 ) : 0
( 1115 , 314 ) : 0
( 314 , 295 ) : 0
( 314 , 333 ) : 0
( 314 , 335 ) : 0
( 315 , 295 ) : 0
( 315 , 335 ) : 0
( 315 , 1116 ) : 0
( 1116 , 336 ) : 0
( 336 , 1117 ) : 0
( 1117 , 315 ) : 0
( 316 , 315 ) : 0
( 316 , 1118 ) : 0
( 1118 , 317 ) : 0
( 317 , 1119 ) : 0
( 1119 , 316 ) : 0
( 316 , 1120 ) : 0
( 1120 , 336 ) : 0
( 336 , 1121 ) : 0
( 1121 , 316 ) : 0
( 316 , 337 ) : 0
( 317 , 1122 ) : 0
( 1122 , 338 ) : 0
( 338 , 1123 ) : 0
( 1123 , 317 ) : 0
( 318 , 317 ) : 0
( 318 , 1124 ) : 0
( 1124 , 338 ) : 0
( 338 , 1125 ) : 0
( 1125 , 318 ) : 0
( 318 , 337 ) : 0
( 319 , 299 ) : 0
( 319 , 318 ) : 0
( 319 , 339 ) : 0
( 320 , 321 ) : 0
( 320 , 301 ) : 0
( 321 , 1126 ) : 0
( 1126 , 341 ) : 0
( 341 , 1127 ) : 0
( 1127 , 321 ) : 0
( 321 , 1128 ) : 507
( 1128 , 340 ) : 507
( 340 , 1129 ) : 0
( 1

( 42 , 21 ) : 0
( 2 , 22 ) : 0
( 23 , 22 ) : 0
( 448 , 22 ) : 0
( 42 , 448 ) : 0
( 449 , 42 ) : 0
( 22 , 449 ) : 0
( 450 , 22 ) : 0
( 43 , 450 ) : 0
( 451 , 43 ) : 0
( 22 , 451 ) : 0
( 452 , 23 ) : 0
( 43 , 452 ) : 0
( 453 , 43 ) : 0
( 23 , 453 ) : 0
( 2 , 23 ) : 0
( 454 , 23 ) : 0
( 42 , 454 ) : 0
( 455 , 42 ) : 0
( 23 , 455 ) : 0
( 25 , 24 ) : 0
( 456 , 24 ) : 0
( 44 , 456 ) : 0
( 457 , 44 ) : 0
( 24 , 457 ) : 0
( 458 , 24 ) : 0
( 43 , 458 ) : 0
( 459 , 43 ) : 0
( 24 , 459 ) : 0
( 5 , 25 ) : 0
( 460 , 25 ) : 0
( 45 , 460 ) : 0
( 461 , 45 ) : 0
( 25 , 461 ) : 0
( 462 , 25 ) : 0
( 46 , 462 ) : 0
( 463 , 46 ) : 0
( 25 , 463 ) : 0
( 6 , 26 ) : 0
( 464 , 26 ) : 0
( 46 , 464 ) : 0
( 465 , 46 ) : 0
( 26 , 465 ) : 0
( 466 , 26 ) : 0
( 47 , 466 ) : 0
( 467 , 47 ) : 0
( 26 , 467 ) : 0
( 26 , 27 ) : 0
( 6 , 27 ) : 0
( 468 , 28 ) : 0
( 47 , 468 ) : 0
( 469 , 47 ) : 0
( 28 , 469 ) : 0
( 49 , 28 ) : 0
( 9 , 29 ) : 0
( 28 , 29 ) : 0
( 8 , 29 ) : 0
( 10 , 30 ) : 0
( 29 , 30 ) : 0
( 470 , 30 ) : 0
( 

( 631 , 95 ) : 0
( 94 , 631 ) : 0
( 114 , 94 ) : 0
( 73 , 94 ) : 0
( 75 , 94 ) : 0
( 113 , 94 ) : 0
( 632 , 95 ) : 0
( 96 , 632 ) : 0
( 633 , 96 ) : 0
( 95 , 633 ) : 0
( 115 , 95 ) : 0
( 634 , 95 ) : 0
( 116 , 634 ) : 0
( 635 , 116 ) : 0
( 95 , 635 ) : 0
( 636 , 97 ) : 0
( 98 , 636 ) : 0
( 637 , 98 ) : 0
( 97 , 637 ) : 0
( 116 , 97 ) : 0
( 79 , 98 ) : 0
( 117 , 98 ) : 0
( 98 , 99 ) : 0
( 638 , 99 ) : 0
( 119 , 638 ) : 0
( 639 , 119 ) : 0
( 99 , 639 ) : 0
( 120 , 100 ) : 0
( 81 , 100 ) : 0
( 82 , 101 ) : 0
( 640 , 101 ) : 0
( 120 , 640 ) : 0
( 641 , 120 ) : 0
( 101 , 641 ) : 0
( 122 , 101 ) : 0
( 101 , 102 ) : 0
( 103 , 102 ) : 0
( 642 , 102 ) : 0
( 122 , 642 ) : 0
( 643 , 122 ) : 0
( 102 , 643 ) : 0
( 83 , 102 ) : 0
( 644 , 102 ) : 0
( 121 , 644 ) : 0
( 645 , 121 ) : 0
( 102 , 645 ) : 0
( 123 , 102 ) : 0
( 83 , 103 ) : 0
( 104 , 103 ) : 0
( 646 , 103 ) : 0
( 123 , 646 ) : 0
( 647 , 123 ) : 0
( 103 , 647 ) : 0
( 82 , 103 ) : 0
( 124 , 103 ) : 0
( 105 , 104 ) : 0
( 124 , 104 ) : 0
( 85 ,

( 924 , 229 ) : 0
( 248 , 924 ) : 0
( 925 , 248 ) : 0
( 229 , 925 ) : 0
( 926 , 230 ) : 0
( 231 , 926 ) : 0
( 927 , 231 ) : 0
( 230 , 927 ) : 0
( 928 , 230 ) : 0
( 250 , 928 ) : 0
( 929 , 250 ) : 0
( 230 , 929 ) : 0
( 211 , 230 ) : 0
( 930 , 230 ) : 0
( 251 , 930 ) : 0
( 931 , 251 ) : 0
( 230 , 931 ) : 0
( 251 , 231 ) : 0
( 932 , 231 ) : 0
( 250 , 932 ) : 0
( 933 , 250 ) : 0
( 231 , 933 ) : 0
( 252 , 231 ) : 0
( 231 , 232 ) : 0
( 934 , 232 ) : 0
( 252 , 934 ) : 0
( 935 , 252 ) : 0
( 232 , 935 ) : 0
( 232 , 233 ) : 0
( 234 , 233 ) : 0
( 936 , 233 ) : 0
( 253 , 936 ) : 0
( 937 , 253 ) : 0
( 233 , 937 ) : 0
( 252 , 233 ) : 0
( 938 , 234 ) : 0
( 254 , 938 ) : 0
( 939 , 254 ) : 0
( 234 , 939 ) : 0
( 215 , 234 ) : 0
( 940 , 234 ) : 0
( 253 , 940 ) : 0
( 941 , 253 ) : 0
( 234 , 941 ) : 0
( 255 , 234 ) : 0
( 215 , 235 ) : 0
( 234 , 235 ) : 0
( 942 , 235 ) : 0
( 236 , 942 ) : 0
( 943 , 236 ) : 0
( 235 , 943 ) : 0
( 944 , 235 ) : 0
( 255 , 944 ) : 0
( 945 , 255 ) : 0
( 235 , 945 ) : 0
( 214 , 23

( 348 , 1201 ) : 0
( 1202 , 348 ) : 0
( 367 , 1202 ) : 0
( 1203 , 367 ) : 0
( 348 , 1203 ) : 0
( 1204 , 348 ) : 0
( 369 , 1204 ) : 0
( 1205 , 369 ) : 0
( 348 , 1205 ) : 0
( 329 , 349 ) : 0
( 369 , 349 ) : 0
( 328 , 349 ) : 0
( 330 , 349 ) : 0
( 368 , 349 ) : 0
( 349 , 350 ) : 0
( 351 , 350 ) : 0
( 1206 , 350 ) : 0
( 370 , 1206 ) : 0
( 1207 , 370 ) : 0
( 350 , 1207 ) : 0
( 329 , 350 ) : 0
( 1208 , 350 ) : 0
( 371 , 1208 ) : 0
( 1209 , 371 ) : 0
( 350 , 1209 ) : 0
( 331 , 351 ) : 0
( 352 , 351 ) : 0
( 1210 , 351 ) : 0
( 371 , 1210 ) : 0
( 1211 , 371 ) : 0
( 351 , 1211 ) : 0
( 353 , 352 ) : 0
( 371 , 352 ) : 0
( 1212 , 353 ) : 0
( 373 , 1212 ) : 0
( 1213 , 373 ) : 0
( 353 , 1213 ) : 0
( 334 , 353 ) : 0
( 372 , 353 ) : 0
( 374 , 353 ) : 0
( 334 , 354 ) : 0
( 353 , 354 ) : 0
( 1214 , 354 ) : 0
( 374 , 1214 ) : 0
( 1215 , 374 ) : 0
( 354 , 1215 ) : 0
( 333 , 354 ) : 0
( 354 , 355 ) : 0
( 356 , 355 ) : 0
( 1216 , 355 ) : 0
( 375 , 1216 ) : 0
( 1217 , 375 ) : 0
( 355 , 1217 ) : 0
( 1218 , 355 